# Assignment 2
## Task 2:
*Perform hyperparameter tuning of the number of attention heads and try a deeper embedding of the node features.*

*For the attention heads, just evaluate 2 additional settings.*

*For the embedding of the node features, instead of the linear transformation of the node states as suggested in the tutorial, try to add one fully connected layer with ReLU activation and one additional fully connected layer.*

---

#### Imports & Setup

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

>   - The import of `matplotlib.pyplot` (`plt`) is new compared to Task 1.
>   - It suggests that some form of **visualization (e.g., metric evolution or comparisons)** may be performed later — likely useful for **hyperparameter tuning analysis**.
>   - All other imports are unchanged and reused from Task 1.

#### Data Aggregation from Multiple Scenes

In [ ]:
# Directory containing .nodes and .edges files
DATASET_PATH = "dataset"

# Get all scene IDs (each scene = a file pair: .nodes + .edges)
scene_ids = sorted(set(f.split(".")[0] for f in os.listdir(DATASET_PATH)))

# Lists to store combined node and edge data
all_nodes = []
all_edges = []
offset = 0 # Offset to make node IDs unique across all scenes

for scene_id in scene_ids:
    nodes_path = os.path.join(DATASET_PATH, f"{scene_id}.nodes")
    edges_path = os.path.join(DATASET_PATH, f"{scene_id}.edges")
    
    if not os.path.exists(nodes_path) or not os.path.exists(edges_path):
        continue # Skip if files are missing
    
    # Load node attributes
    nodes_df = pd.read_csv(
        nodes_path, header=None,
        names=["node_id", "current_x", "current_y", "prev_x", "prev_y", "future_x", "future_y"],
        na_values=["_", "NA", "NaN", "nan"]
    )
    
    # Remove rows with missing position data (except for future positions)
    nodes_df = nodes_df.dropna(subset=["current_x", "current_y", "prev_x", "prev_y"])
    nodes_df[["current_x", "current_y", "prev_x", "prev_y"]] = nodes_df[["current_x", "current_y", "prev_x", "prev_y"]].astype(float)
    
    # Reindex local node IDs into global unique IDs
    local_to_global = {nid: i + offset for i, nid in enumerate(nodes_df["node_id"])}
    nodes_df["global_id"] = nodes_df["node_id"].map(local_to_global)
    
    # Load and remap edge list to global IDs
    edges_df = pd.read_csv(edges_path, header=None, names=["target", "source"])
    edges_df["source"] = edges_df["source"].map(local_to_global)
    edges_df["target"] = edges_df["target"].map(local_to_global)
    
    # Duplicate edges in the opposite direction to make graph undirected
    reversed_edges = edges_df.rename(columns={"source": "target", "target": "source"})
    full_edges = pd.concat([edges_df, reversed_edges])
    
    # Store the processed scene data
    all_nodes.append(nodes_df)
    all_edges.append(full_edges)

    # Update offset for next scene
    offset += len(nodes_df)
    
# Concatenate all scenes into single global DataFrames
nodes_all = pd.concat(all_nodes).sort_values("global_id")
edges_all = pd.concat(all_edges).dropna().astype(int)

#### Tensor Creation for Model Input

In [ ]:
# Extract node input features as float tensor (shape: [num_nodes, 4])
node_features = tf.convert_to_tensor(
    nodes_all[["current_x", "current_y", "prev_x", "prev_y"]].to_numpy(),
    dtype=tf.float32
)

# Extract labels (future positions), replacing NaN with 0.0 to avoid tensor errors
labels = tf.convert_to_tensor(
    nodes_all[["future_x", "future_y"]].fillna(0.0).to_numpy(),
    dtype=tf.float32
)

# Create a mask to mark which nodes have valid future positions (used for loss filtering)
mask = tf.convert_to_tensor(
    ~nodes_all[["future_x", "future_y"]].isna().any(axis=1),
    dtype=tf.bool
)

# Create edge list as tensor (shape: [num_edges, 2]), each row: [target, source]
edges = tf.convert_to_tensor(
    edges_all[["target", "source"]].to_numpy(),
    dtype=tf.int64
)

#### Train-Test Split

In [ ]:
# Shuffle all node indices and split 80/20 for training and testing
split_ratio = 0.8
random_indices = np.random.permutation(len(nodes_all)) # Randomly permuted indices
split_idx = int(split_ratio * len(random_indices))
train_indices = random_indices[: split_idx]
test_indices = random_indices[split_idx :]

# Select node features and labels using the indices (converted to NumPy first)
train_node_features = node_features.numpy()[train_indices]
test_node_features = node_features.numpy()[test_indices]

train_labels = labels.numpy()[train_indices]
test_labels = labels.numpy()[test_indices]

train_mask = mask.numpy()[train_indices]
test_mask = mask.numpy()[test_indices]

# Convert everything back to TensorFlow tensors
train_node_features = tf.convert_to_tensor(train_node_features, dtype=tf.float32)
test_node_features = tf.convert_to_tensor(test_node_features, dtype=tf.float32)

train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.float32)

train_mask = tf.convert_to_tensor(train_mask, dtype=tf.bool)
test_mask = tf.convert_to_tensor(test_mask, dtype=tf.bool)

*Display*:

In [ ]:
print("Shape of training features:", train_node_features.shape)
print("Shape of training labels:", train_labels.shape)
print("Shape of training masks:", train_mask.shape)

print("Shape of test features:", test_node_features.shape)
print("Shape of test labels:", test_labels.shape)
print("Shape of test masks:", test_mask.shape)

#### Graph Attention Layer Definition (GAT Layer)

In [ ]:
class GraphAttention(layers.Layer):
    def __init__(
        self,
        units,
        kernel_initializer="glorot_uniform",
        kernel_regularizer=None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.units = units
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
    
    def build(self, input_shape):
        # input_shape[0] is (num_nodes, feature_dim)
        self.kernel = self.add_weight(
            shape=(input_shape[0][-1], self.units),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel",
            trainable=True,
        )
        # attention kernel takes concatenated pair [h_i || h_j]
        self.kernel_attention = self.add_weight(
            shape=(self.units * 2, 1),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            name="kernel_attention",
            trainable=True,
        )
        super().build(input_shape)
        
    def call(self, inputs):
        node_states, edges = inputs

        # 1) Linear transform
        h = tf.matmul(node_states, self.kernel)  # (N, units)

        # 2) Compute attention scores for each edge
        #    gather [h_target, h_source]
        edge_states = tf.gather(h, edges)            # (E, 2, units)
        edge_states = tf.reshape(
            edge_states, (tf.shape(edges)[0], 2 * self.units)
        )
        scores = tf.nn.leaky_relu(
            tf.matmul(edge_states, self.kernel_attention)
        )  # (E,1)
        scores = tf.squeeze(scores, -1)               # (E,)
        
        # 3) Normalize per target node
        scores_exp = tf.exp(tf.clip_by_value(scores, -2, 2))
        denom = tf.math.unsorted_segment_sum(
            data=scores_exp,
            segment_ids=edges[:, 0],
            num_segments=tf.shape(node_states)[0]
        )
        # broadcast denom back to each edge
        denom_per_edge = tf.gather(denom, edges[:, 0])
        alpha = scores_exp / (denom_per_edge + tf.keras.backend.epsilon())
        
        # 4) Weighted aggregation of neighbor states
        neigh = tf.gather(h, edges[:, 1])             # (E, units)
        out = tf.math.unsorted_segment_sum(
            data=neigh * alpha[:, tf.newaxis],
            segment_ids=edges[:, 0],
            num_segments=tf.shape(node_states)[0]
        )
        return out

#### Define Multi-Head Attention Layer

In [ ]:
class MultiHeadGraphAttention(layers.Layer):
    def __init__(self, units, num_heads=8, merge_type="concat", **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.merge_type = merge_type
        # create one GraphAttention per head
        self.attention_heads = [GraphAttention(units) for _ in range(num_heads)]

    def call(self, inputs):
        node_states, edges = inputs
        head_outputs = [head([node_states, edges]) for head in self.attention_heads]
        if self.merge_type == "concat":
            h = tf.concat(head_outputs, axis=-1)  # (N, units * num_heads)
        else:
            h = tf.reduce_mean(tf.stack(head_outputs, axis=-1), axis=-1)  # (N, units)
        return tf.nn.relu(h)

#### Define Graph Attention Network (GAT) Model

In [ ]:
class GraphAttentionNetwork(keras.Model):
    def __init__(
        self,
        node_features,
        edges,
        hidden_units=32,
        num_heads=4,
        num_layers=2,
        output_dim=2,
        **kwargs
    ):
        super().__init__(**kwargs)
        # Store fixed graph structure
        self.node_features = node_features
        self.edges = edges

        # Deeper transformation of node features using two fully connected layers
        self.preprocess = keras.Sequential([
            layers.Dense(hidden_units * num_heads, activation="relu", name="embedding_fc1"),
            layers.Dense(hidden_units * num_heads, name="embedding_fc2"),
        ], name="node_embedding")

        # Stack multiple multi-head GAT layers
        self.gat_layers = [
            MultiHeadGraphAttention(hidden_units, num_heads, merge_type="concat")
            for _ in range(num_layers)
        ]

        # Output layer: regresses to (x, y)
        self.output_layer = layers.Dense(output_dim)

    def call(self, inputs):
        x, edges = inputs

        # Initial feature transformation
        x = self.preprocess(x)

        # Apply GAT layers with residual connections
        for gat in self.gat_layers:
            x = gat([x, edges]) + x
        
        # Final output layer for (x, y) coordinates
        return self.output_layer(x)

    def train_step(self, data):
        indices, y_true = data

        # Custom training loop with batch sampling
        with tf.GradientTape() as tape:
            y_pred = self([self.node_features, self.edges])
            y_pred_batch = tf.gather(y_pred, indices)   # Select batch samples
            loss = self.compiled_loss(y_true, y_pred_batch)

        # Compute gradients and update weights
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

        # Track metrics
        self.compiled_metrics.update_state(y_true, y_pred_batch)
        return {m.name: m.result() for m in self.metrics}

    def test_step(self, data):
        indices, y_true = data
        y_pred = self([self.node_features, self.edges])
        y_pred_batch = tf.gather(y_pred, indices)
        loss = self.compiled_loss(y_true, y_pred_batch)
        self.compiled_metrics.update_state(y_true, y_pred_batch)
        return {m.name: m.result() for m in self.metrics}

    def predict_step(self, data):
        indices = data
        y_pred = self([self.node_features, self.edges])
        return tf.gather(y_pred, indices)

>   - **Purpose**: Improves the model’s capacity to capture nonlinear feature interactions prior to message passing.
>   - **Details**:
>       - The first `Dense` layer applies a ReLU-activated transformation to introduce non-linearity and richer representations.
>       - The second `Dense` layer serves to consolidate and project the features to the same dimensionality expected by the GAT layers.
>       - This structure increases model expressiveness and supports better downstream attention computations.

#### Define Training Configuration and Hyperparameter Search Space

In [ ]:
# ----------- Hyperparameters -----------
HIDDEN_UNITS = 32       # size of each GAT attention head
NUM_LAYERS = 2          # number of stacked GAT layers
OUTPUT_DIM = 2          # we predict 2 values: (future_x, future_y)

NUM_EPOCHS = 100
BATCH_SIZE = 256
VALIDATION_SPLIT = 0.1  # 10% of train data used for validation
LEARNING_RATE = 1e-4
PATIENCE = 30

HEAD_OPTIONS = [2, 4, 8]

# ----------- Loss and optimizer -----------
loss_fn = keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

# ----------- Early stopping to prevent overfitting -----------
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_mae",          # monitor validation MAE
    min_delta=1e-4,             # minimal change to be considered improvement
    patience=PATIENCE,          # stop if no improvement over X epochs
    restore_best_weights=True
)

>   - `HEAD_OPTIONS = [2, 4, 8]` defines the hyperparameter tuning space for attention heads.
>   - The node feature embedding uses **two Dense layers** instead of a single linear projection.
>   - `HIDDEN_UNITS`, `NUM_LAYERS`, and `OUTPUT_DIM` define GAT model depth and output shape.
>   - `LEARNING_RATE = 1e-4` is lower than usual to support deeper, more expressive architectures.
>   - `EarlyStopping` is more tolerant (`patience=30`) to accommodate slower convergence.
>   - Loss: `MeanSquaredError`, suitable for regression; Metric: `MeanAbsoluteError`, interpretable.
>   - This section **does not compile or train** models yet — it only sets up the config space.

#### Train and Compare GAT Models with Varying Attention Heads

In [ ]:
histories = {}
mae_results = {}
predictions = {}

for num_heads in HEAD_OPTIONS:
    print(f"\n--- Training model with {num_heads} attention heads ---")
    
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)

    # Initialize a GAT model with the given number of attention heads
    gat_model = GraphAttentionNetwork(
        node_features=node_features,
        edges=edges,
        hidden_units=HIDDEN_UNITS,
        num_heads=num_heads,
        num_layers=NUM_LAYERS,
        output_dim=OUTPUT_DIM
    )

    # Compile with MSE loss and MAE metric
    gat_model.compile(
        loss=loss_fn,
        optimizer=optimizer,
        metrics=[mae_metric]
    )

    # Train the model on node indices (mini-batch), full graph is fixed
    history = gat_model.fit(
        x=train_indices,
        y=train_labels,
        validation_split=VALIDATION_SPLIT,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        callbacks=[early_stopping],
        verbose=0,
    )

    # Evaluate performance on test set
    loss, mae = gat_model.evaluate(
        x=test_indices,
        y=test_labels,
        verbose=0
    )

    # Store results for comparison
    histories[num_heads] = history.history
    mae_results[num_heads] = mae
    predictions[num_heads] = gat_model.predict(x=test_indices)

# Final predictions correspond to the last model trained (with highest num_heads)
test_preds = gat_model.predict(x=test_indices)

#### Validation MAE Curves — Attention Head Comparison

In [ ]:
# MAE curves on the validation set (to compare convergence behaviour)plt.figure(figsize=(10, 6))
for num_heads in HEAD_OPTIONS:
    val_mae = histories[num_heads]['val_mae']
    plt.plot(val_mae, label=f"{num_heads} heads")
plt.title("Validation MAE per number of attention heads")
plt.xlabel("Epoch")
plt.ylabel("MAE")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

>   - This plot compares **how each model configuration (based on number of attention heads)** performs during training, using the **validation MAE** as the main reference.
>   - **MAE (Mean Absolute Error)** measures the **average absolute difference between predicted and true values**. In this case, it reflects the **average distance error (in units) between the predicted and actual future positions** of nodes (i.e., agents).
>   - A lower MAE means more **accurate trajectory prediction**, while a high MAE indicates the model is either underfitting or struggling with generalization.
>   - The plot helps assess whether increasing the number of attention heads improves predictive performance or leads to overfitting.
>   - A **faster MAE decrease and lower final MAE** are signs of better convergence and more effective learning dynamics.

#### Test MAE Comparison by Attention Head Count

In [ ]:
# Bar chart summarizing test MAE for each model (by number of attention heads)
plt.figure(figsize=(6, 5))
plt.bar(
    [str(h) + " heads" for h in mae_results.keys()],
    [mae_results[h] for h in mae_results],
    color="skyblue"
)
plt.title("Test MAE per model")
plt.ylabel("MAE")
plt.grid(axis="y")
plt.tight_layout()
plt.show()

>   - This bar chart summarizes the **final model performance on the test set**, after training with different numbers of attention heads.
>   - It displays the **Mean Absolute Error (MAE)** on **unseen test nodes**, providing an estimate of **generalization capability**.
>   - Each bar corresponds to a trained GAT model using a specific number of attention heads (e.g., 2, 4, or 8).
>   - Lower MAE values indicate **more accurate future position predictions**.
>   - This comparison is useful to:
>       - Validate the benefit (or not) of using more attention heads.
>       - Detect overfitting if higher-capacity models (more heads) don’t improve or worsen test MAE.

*Display*:

#### Prediction Samples from Best Model

In [ ]:
# Visualization of a few predictions vs ground truth for the best model
best_heads = min(mae_results, key=mae_results.get)
best_preds = predictions[best_heads]

print(f"\nBest model: {best_heads} attention heads (Test MAE = {mae_results[best_heads]:.4f})")

# Initialize the list for storing error data
errors = []

# Loop through the predictions and compute the error
for i, (pred, true) in enumerate(zip(best_preds, test_labels)):
    true_norm = tf.norm(true).numpy()  # L2 norm of ground truth
    l2_error = tf.norm(pred - true).numpy()  # L2 error between prediction and ground truth
    errors.append({
        "index": i,
        "prediction": pred,
        "ground_truth": true,
        "l2_error": l2_error
    })

# Sort the errors in ascending order of L2 error (smallest error first)
errors_sorted = sorted(errors, key=lambda x: x["l2_error"], reverse=False)

# Display the top 5 examples with the smallest errors
num_examples = 5
for i, item in enumerate(errors_sorted[:num_examples]):
    pred = item["prediction"]
    true = item["ground_truth"]
    l2_error = item["l2_error"]
    print(f"Example {i+1}:")
    print(f"\tPrediction   = ({pred[0]:.2f}, {pred[1]:.2f})")
    print(f"\tGround Truth = ({true[0]:.2f}, {true[1]:.2f})")
    print(f"\tL2 Error     = {l2_error:.2f} units")
    print("---" * 20)